In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [2]:
test_matrix = np.array([[1, 2], [3, 4]])
matr2 = np.array([[5, 6], [7, 8]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [2]:
def kron(A, B):
    return la.kron(A, B)

In [4]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(np.eye(A.shape[0]), B)
    
    return L+R

In [5]:
print(kronsum(test_matrix, matr2))

[[ 6.00000000  6.00000000  2.00000000  0.00000000]
 [ 7.00000000  9.00000000  0.00000000  2.00000000]
 [ 3.00000000  0.00000000  9.00000000  6.00000000]
 [ 0.00000000  3.00000000  7.00000000  12.00000000]]


In [6]:
print(kronsum(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 2.00000000  2.00000000  2.00000000  0.00000000]
 [ 3.00000000  5.00000000  0.00000000  2.00000000]
 [ 3.00000000  0.00000000  5.00000000  2.00000000]
 [ 0.00000000  3.00000000  3.00000000  8.00000000]]


In [7]:
n = 3
p_num = 100
eps_G = 10 ** (-8)
eps_Phi = 10 ** (-6)
eps_p_i = 10 ** (-6)

In [8]:
# Входной BMAP
matrD_0 = np.array([[-86., 0.01], [0.02, -2.76]]) / 7
matrD = np.array([[85, 0.99], [0.2, 2.54]]) / 7
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

for matr in matrD_k:
    print(matr)

[[-12.28571429  0.00142857]
 [ 0.00285714 -0.39428571]]
[[ 4.97658080  0.05796253]
 [ 0.01170960  0.14871194]]
[[ 3.98126464  0.04637002]
 [ 0.00936768  0.11896956]]
[[ 3.18501171  0.03709602]
 [ 0.00749415  0.09517564]]


In [9]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
print(matrD_1_)
matr_a = copy.deepcopy(matrD_1_)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)

matrdD_1_ = copy.deepcopy(matrD_k[1])

print(theta)

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = np.dot(np.dot(theta, matrdD_1_), vect_e)
print(lamD)

[[-0.14285714  0.14285714]
 [ 0.03142857 -0.03142857]]
[ 0.18032787  0.81967213]
[ 4.69791416]


In [10]:
# Поток поломок MAP
matrH0 = np.array([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.array([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = np.dot(np.dot(gamma, matrH1), vect_e)
print(h)

[[-0.05392500  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  1.00000000]
 [ 0.05392500 -0.14662500]]
[ 6.00065225]


In [11]:
# Поток обслуживания PH1
beta1 = np.matrix([1, 0])
matrS1 = np.matrix([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1], [1]])
matrS1_0 = - matrS1 * M1_e
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
print(np.dot(beta1, la.inv(matrS1)))
mu_1 = -la.inv(np.dot(np.dot(beta1, la.inv(matrS1)), vect_e))
print(mu_1)

[[-0.05000000 -0.05000000]]
[[ 10.00000000]]


In [12]:
# Поток обслуживания PH2
beta2 = np.matrix([1, 0])
matrS2 = np.matrix([[-2, 2], [0, -2]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1], [1]])
matrS2_0 = np.dot(- matrS2, M2_e)

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(np.dot(np.dot(beta2, la.inv(matrS2)), vect_e))
print(mu_2)

[[ 1.00000000]]


In [13]:
matrS_w = kron(np.dot(matrS1_0, beta1), 
               np.dot(M2_e, beta2)) + kron(np.dot(M1_e, beta1), np.dot(matrS2_0, beta2))

In [14]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([0.05, 0.95])
matrA1 = np.matrix([[-1.86075, 0.], [0., -146.9994]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1], [1]])
matrA1_0 = - np.dot(matrA1, L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(np.dot(np.dot(alpha1, la.inv(matrA1)), vect_e))
print(kappa_1)

[[ 29.99985287]]


In [15]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([0.05, 0.95])
matrA2 = np.matrix([[-5.58225, 0.], [0., -440.9982]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1], [1]])
matrA2_0 = - np.dot(matrA2, L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(np.dot(np.dot(alpha2, la.inv(matrA2)), vect_e))
print(kappa_2)

[[ 89.99955862]]


In [16]:
# Поток ремонта PH
tau = np.matrix([0.98, 0.02])
matrT = np.matrix([[-100., 0.], [0., -0.0002]])
T_e = np.matrix([[1], [1]])
matrT0 = - np.dot(matrT, T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -np.dot(np.dot(tau, la.inv(matrT)), vect_e)
print(phi)

[[ 100.00980000]]


In [17]:
a = W_ * V_

In [20]:
print(kronsum(matrD_k[0], matrH0))

[[-20.39643929  0.00000000  0.00142857  0.00000000]
 [ 0.00000000 -12.54896429  0.00000000  0.00142857]
 [ 0.00285714  0.00000000 -8.50501071  0.00000000]
 [ 0.00000000  0.00285714  0.00000000 -0.65753571]]


In [21]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0)

36
[[-20.39643929  0.00000000  0.00142857  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.39478320  7.50088080  0.00805680  0.15307920
   0.00264233  0.05020417  0.00005393  0.00102458  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00000000 -12.54896429  0.00000000  0.00142857  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00718463  0.13650788  0.00014663  0.00278588
   0.00571463  0.10857788  0.00011663  0.00221588  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00285714  0.00000000 -8.50501071  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00

In [22]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)

for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    print('matrQw_k[' + str(i) + '] = ', sum)

matrQw_k[0] =  8.881784197e-16
matrQw_k[1] =  8.881784197e-16
matrQw_k[2] =  5.55111512313e-16
matrQw_k[3] =  1.11022302463e-16
matrQw_k[4] =  -4.4408920985e-15
matrQw_k[5] =  -8.881784197e-16
matrQw_k[6] =  2.6645352591e-15
matrQw_k[7] =  -5.77315972805e-14
matrQw_k[8] =  2.22044604925e-16
matrQw_k[9] =  -1.487698853e-14
matrQw_k[10] =  0.0
matrQw_k[11] =  1.42108547152e-14
matrQw_k[12] =  8.881784197e-16
matrQw_k[13] =  -8.881784197e-16
matrQw_k[14] =  -6.2172489379e-15
matrQw_k[15] =  -8.881784197e-16
matrQw_k[16] =  6.88338275268e-15
matrQw_k[17] =  6.10622663544e-16
matrQw_k[18] =  -8.881784197e-16
matrQw_k[19] =  1.11022302463e-16
matrQw_k[20] =  1.33226762955e-14
matrQw_k[21] =  -1.50990331349e-14
matrQw_k[22] =  8.881784197e-16
matrQw_k[23] =  8.881784197e-16
matrQw_k[24] =  -2.6645352591e-15
matrQw_k[25] =  7.9936057773e-15
matrQw_k[26] =  -8.881784197e-16
matrQw_k[27] =  7.9936057773e-15
matrQw_k[28] =  2.27595720048e-15
matrQw_k[29] =  3.06976666309e-14
matrQw_k[30] =  4.996

In [23]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)

print(matrQv_0)

[[ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 20.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.000000

In [24]:
# Q_0
block0 = kron(np.eye(a), np.dot(matrS1_0, beta1))
block1 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)

print(matrQ_0)

[[ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000]
 [ 20.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000

In [25]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])

print(matrQ_1)

[[-40.39643929  20.00000000  0.00000000  0.00000000  0.00142857  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.39478320  7.50088080
   0.00805680  0.15307920  0.00264233  0.05020417  0.00005393  0.00102458
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000]
 [ 0.00000000 -40.39643929  0.00000000  0.00000000  0.00000000  0.00142857
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.000

In [26]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+2):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for j in range(1, np.array(matrQ_k).shape[0]):
        sum += np.sum(matrQ_k[j][i])
    print('matrQ_k[' + str(i) + '] = ', sum)

matrQ_k[0] =  8.881784197e-16
matrQ_k[1] =  -7.9936057773e-15
matrQ_k[2] =  2.6645352591e-15
matrQ_k[3] =  -8.881784197e-16
matrQ_k[4] =  5.55111512313e-17
matrQ_k[5] =  -1.72084568817e-15
matrQ_k[6] =  1.49880108324e-15
matrQ_k[7] =  1.83186799063e-15
matrQ_k[8] =  8.881784197e-16
matrQ_k[9] =  6.2172489379e-15
matrQ_k[10] =  -2.6645352591e-15
matrQ_k[11] =  6.2172489379e-15
matrQ_k[12] =  4.4408920985e-15
matrQ_k[13] =  -5.77315972805e-14
matrQ_k[14] =  4.4408920985e-15
matrQ_k[15] =  -5.77315972805e-14
matrQ_k[16] =  -8.32667268469e-16
matrQ_k[17] =  -3.36952687974e-14
matrQ_k[18] =  -1.72084568817e-15
matrQ_k[19] =  -3.36952687974e-14
matrQ_k[20] =  -1.66533453694e-16
matrQ_k[21] =  -5.49560397189e-15
matrQ_k[22] =  5.55111512313e-17
matrQ_k[23] =  -5.27355936697e-15
matrQ_k[24] =  6.2172489379e-15
matrQ_k[25] =  -8.881784197e-16
matrQ_k[26] =  6.2172489379e-15
matrQ_k[27] =  -8.881784197e-16
matrQ_k[28] =  -8.881784197e-16
matrQ_k[29] =  -8.881784197e-16
matrQ_k[30] =  -8.88178419

In [26]:
# Проверка генератора Q

In [28]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])


def calc_G(matrG_prev):
    temp_sum = copy.deepcopy(matrQ_k[0]) 
    for k in range(2, n + 2):
        temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG_prev, k))
    matrG_new = np.dot(matrQ_1_neg_inv, temp_sum)
    return matrG_new

matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)


i = 1
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    i += 1

pd.DataFrame(matrG)
print(la.norm(matrG, ord=np.inf))

5 norm s =  16.110538641686208
5 norm s =  16.110538641686208
5 norm s =  16.110538641686198
5 norm s =  16.110538641686194
5 norm s =  16.11053864168621
5 norm s =  16.11053864168624
5 norm s =  16.110538641686247
5 norm s =  16.11053864168629
5 norm s =  16.11053864168631
5 norm s =  16.11053864168636
5 norm s =  16.110538641686414
5 norm s =  16.11053864168646
5 norm s =  16.110538641686514
5 norm s =  16.110538641686567
5 norm s =  16.110538641686638
5 norm s =  16.110538641686713
5 norm s =  16.11053864168681
5 norm s =  16.110538641686887
5 norm s =  16.110538641686993
5 norm s =  16.1105386416871
5 norm s =  16.110538641687228
5 norm s =  16.11053864168738
5 norm s =  16.11053864168754
5 norm s =  16.110538641687707
5 norm s =  16.110538641687903
5 norm s =  16.110538641688102
5 norm s =  16.110538641688322
5 norm s =  16.110538641688574
5 norm s =  16.110538641688855
5 norm s =  16.11053864168917
5 norm s =  16.110538641689505
5 norm s =  16.11053864168988
5 norm s =  16.110538

5 norm s =  16.11053864172234
5 norm s =  16.11053864172419
5 norm s =  16.110538641726038
5 norm s =  16.11053864172791
5 norm s =  16.11053864172979
5 norm s =  16.110538641731694
5 norm s =  16.110538641733598
5 norm s =  16.11053864173552
5 norm s =  16.110538641737435
5 norm s =  16.110538641739378
5 norm s =  16.110538641741314
5 norm s =  16.11053864174326
5 norm s =  16.11053864174521
5 norm s =  16.110538641747162
5 norm s =  16.110538641749113
5 norm s =  16.11053864175108
5 norm s =  16.11053864175306
5 norm s =  16.110538641755046
5 norm s =  16.11053864175704
5 norm s =  16.110538641759046
5 norm s =  16.11053864176105
5 norm s =  16.110538641763075
5 norm s =  16.110538641765118
5 norm s =  16.110538641767167
5 norm s =  16.110538641769235
5 norm s =  16.110538641771328
5 norm s =  16.110538641773445
5 norm s =  16.110538641775587
5 norm s =  16.11053864177777
5 norm s =  16.110538641779954
5 norm s =  16.110538641782156
5 norm s =  16.110538641784387
5 norm s =  16.11053

5 norm s =  16.11053864188418
5 norm s =  16.11053864188828
5 norm s =  16.110538641892447
5 norm s =  16.110538641896703
5 norm s =  16.11053864190105
5 norm s =  16.11053864190548
5 norm s =  16.110538641909997
5 norm s =  16.110538641914616
5 norm s =  16.110538641919316
5 norm s =  16.110538641924105
5 norm s =  16.110538641929004
5 norm s =  16.110538641934
5 norm s =  16.110538641939083
5 norm s =  16.11053864194429
5 norm s =  16.110538641949585
5 norm s =  16.110538641955003
5 norm s =  16.11053864196052
5 norm s =  16.110538641966176
5 norm s =  16.11053864197191
5 norm s =  16.110538641977772
5 norm s =  16.110538641983748
5 norm s =  16.110538641989844
5 norm s =  16.11053864199609
5 norm s =  16.110538642002442
5 norm s =  16.110538642008922
5 norm s =  16.11053864201552
5 norm s =  16.110538642022284
5 norm s =  16.11053864202916
5 norm s =  16.110538642036193
5 norm s =  16.110538642043366
5 norm s =  16.110538642050674
5 norm s =  16.11053864205814
5 norm s =  16.1105386

5 norm s =  16.110538642521657
5 norm s =  16.110538642538714
5 norm s =  16.110538642556126
5 norm s =  16.11053864257389
5 norm s =  16.11053864259201
5 norm s =  16.11053864261051
5 norm s =  16.11053864262937
5 norm s =  16.11053864264863
5 norm s =  16.11053864266829
5 norm s =  16.110538642688326
5 norm s =  16.1105386427088
5 norm s =  16.11053864272967
5 norm s =  16.110538642750964
5 norm s =  16.110538642772706
5 norm s =  16.11053864279489
5 norm s =  16.11053864281752
5 norm s =  16.110538642840602
5 norm s =  16.110538642864174
5 norm s =  16.110538642888216
5 norm s =  16.110538642912754
5 norm s =  16.11053864293778
5 norm s =  16.11053864296332
5 norm s =  16.110538642989404
5 norm s =  16.110538643016007
5 norm s =  16.110538643043153
5 norm s =  16.11053864307085
5 norm s =  16.1105386430991
5 norm s =  16.110538643127942
5 norm s =  16.11053864315737
5 norm s =  16.110538643187404
5 norm s =  16.110538643218042
5 norm s =  16.110538643249324
5 norm s =  16.1105386432

5 norm s =  16.110538644276623
5 norm s =  16.110538644329502
5 norm s =  16.11053864438346
5 norm s =  16.110538644438517
5 norm s =  16.110538644494692
5 norm s =  16.110538644552022
5 norm s =  16.110538644610507
5 norm s =  16.110538644670193
5 norm s =  16.1105386447311


...

5 norm s =  16.11053866044817
5 norm s =  16.110538660831097
5 norm s =  16.11053866122182
5 norm s =  16.11053866162053
5 norm s =  16.11053866202738


5 norm s =  16.110538662442533
5 norm s =  16.110538662866155
5 norm s =  16.110538663298414
5 norm s =  16.110538663739508
5 norm s =  16.11053866418959


5 norm s =  16.110538664648878
5 norm s =  16.110538665117534
5 norm s =  16.11053866559574
5 norm s =  16.110538666083734
5 norm s =  16.110538666581668


5 norm s =  16.110538667089763
5 norm s =  16.110538667608242
5 norm s =  16.11053866813729
5 norm s =  16.11053866867713
5 norm s =  16.110538669227996


5 norm s =  16.110538669790113
5 norm s =  16.110538670363702
1.0000000017054


In [29]:
la.norm(matrG, ord=np.inf)

1.0000000017054

In [31]:
temp_sum = matrQ_k[1].copy()
for k in range(2, n+2):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = (-matrG_0).dot(matrQv_0)
pd.DataFrame(matrG_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.396833,0.002519,0.268151,0.244523,7.445429e-05,2.186799e-05,1.023466e-06,1.736665e-07,0.000140,1.769089e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.606271,0.001715,0.175245,0.159389,5.161550e-05,1.516101e-05,7.092874e-07,1.203822e-07,0.000092,1.156039e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.004480,0.483810,0.167202,0.289445,2.298020e-06,6.712217e-07,2.023818e-06,3.154654e-07,0.000088,1.116733e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002866,0.695133,0.098474,0.171100,1.484291e-06,4.338016e-07,1.315272e-06,2.050693e-07,0.000052,6.577608e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000589,0.000016,0.968647,0.009691,1.877517e-07,3.480540e-08,4.267502e-09,2.856422e-10,0.000491,6.273390e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000387,0.000009,0.981642,0.005646,1.142628e-07,2.160944e-08,2.560749e-09,1.769470e-10,0.000288,3.687099e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000021,0.000694,0.015557,0.982532,8.357641e-09,1.808865e-09,7.072446e-09,4.638857e-10,0.000016,1.987886e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000011,0.000412,0.007820,0.991163,4.352251e-09,9.521438e-10,3.727694e-09,2.568525e-10,0.000008,1.004572e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.212777,0.003023,0.346482,0.317741,5.306661e-03,1.104153e-04,2.297095e-06,8.226630e-07,0.000183,2.290504e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.387562,0.002553,0.272251,0.248283,7.791672e-05,4.171573e-05,1.067953e-06,1.828495e-07,0.000142,1.796158e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = matrQw_k[l].copy()
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k].copy()
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n+1, p_num):
            matrQ_il[i].append(np.zeros(matrQ_il[i][l % n+1].shape)) 
    else:
        for l in range(0, p_num):
            if l >= i and l - i <= n+1:
                temp_matr = matrQ_k[l-i].copy()
                for k in range(l+1, n+2):
                    if (k - i <= n+1):
                        mult_matr = matrQ_k[k-i].copy()
                        for kk in range(k-1, l-1, -1):
                            if kk == 0:
                                mult_matr = mult_matr.dot(matrG_0)
                            else:
                                mult_matr = mult_matr.dot(matrG)
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))
    
print(len(matrQ_il[3]))
pd.DataFrame(la.inv(-matrQ_il[1][1]))

100


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-1.278757e+11,8.631073e+10,-2.081722e+11,1.985937e+11,5.298922e+11,2.448328e+13,-9.744233e+11,-1.627102e+13,-1.695676e+11,-1.235519e+10,...,-1.124494e+12,-5.904793e+10,1.359566e+12,-5.015837e+11,-3.403660e+12,2.408061e+10,1.844546e+12,1.650937e+12,-2.061435e+12,6.312416e+10
1,-2.507109e+230,-2.372182e+229,-5.254915e+230,5.445672e+230,-1.099653e+232,4.276952e+232,-3.590713e+231,-1.002249e+233,5.571917e+230,-3.763135e+229,...,7.231068e+230,-2.080769e+230,6.139712e+230,-4.261216e+231,-1.378873e+230,3.903817e+228,-4.730562e+231,-2.108453e+231,-1.238400e+232,4.658738e+229
2,-6.892153e+09,2.305044e+09,-9.793501e+09,8.958249e+09,-1.144079e+11,1.139382e+12,-1.353010e+11,-1.098761e+12,-1.385612e+09,-3.252606e+08,...,-2.250941e+10,-3.433800e+09,8.567573e+10,-3.656336e+10,-1.489860e+11,3.788434e+08,1.294128e+11,3.465138e+10,-1.020490e+11,1.860592e+09
3,3.619693e+230,-1.263352e+230,2.981069e+230,-2.509797e+230,7.931023e+231,-3.761986e+232,7.204352e+231,3.788522e+232,-1.250600e+230,-7.977192e+228,...,9.990539e+230,8.391892e+229,-4.183093e+231,8.681346e+230,5.924756e+231,-2.297316e+229,-9.393835e+231,-4.427799e+229,2.166700e+230,-3.369113e+229
4,2.814708e+09,-3.805477e+08,3.019148e+09,-2.623498e+09,1.239844e+11,-3.902097e+11,6.253793e+10,4.394039e+11,-1.366008e+09,-2.335564e+07,...,-2.840367e+09,1.319882e+09,-3.213593e+10,1.784015e+10,4.815326e+10,1.924826e+08,-7.148385e+10,1.481148e+10,4.759486e+10,-3.202024e+08
5,-2.495145e+232,-2.493512e+231,-3.560389e+232,3.177924e+232,-1.523841e+234,4.599143e+234,-1.413361e+234,-5.705286e+234,1.544113e+232,-8.736843e+230,...,9.324798e+232,-1.869745e+232,2.890285e+233,-2.247122e+233,-5.521135e+233,-4.202685e+230,2.674816e+233,-3.402537e+233,-6.897152e+233,2.309761e+231
6,1.627973e+07,-9.435532e+05,1.507450e+07,-1.190435e+07,8.146044e+08,-2.271586e+09,4.147918e+08,2.460582e+09,-7.513445e+06,-8.042099e+05,...,-3.790325e+07,7.566039e+06,-2.661141e+08,7.773937e+07,2.975554e+08,2.546414e+06,-7.275800e+08,6.456087e+07,2.263411e+08,-1.168337e+06
7,2.138406e+232,-1.477630e+232,1.718395e+232,-1.382536e+232,4.080536e+233,-1.901058e+234,-2.548974e+232,2.083735e+234,1.876112e+232,-4.582728e+229,...,6.785154e+232,4.151523e+231,-1.367742e+233,4.854109e+232,4.932402e+233,-3.364333e+231,-3.743465e+233,-4.399304e+232,2.149410e+233,-2.471167e+231
8,1.860708e+12,8.861196e+13,-8.083194e+13,9.395947e+13,7.504544e+15,7.822557e+15,5.125127e+15,3.375467e+15,-2.969278e+14,-1.499080e+13,...,-1.544718e+15,2.442652e+12,-1.513772e+14,5.218879e+14,-1.590244e+15,4.039258e+13,-1.073971e+15,2.779597e+15,-8.535716e+13,6.372805e+13
9,1.788992e+16,1.436766e+16,4.569489e+15,5.888434e+14,2.127985e+18,-1.024194e+18,1.632164e+18,3.221998e+18,-6.435895e+16,-2.726524e+15,...,-3.015886e+17,8.760245e+15,-2.303114e+17,2.209836e+17,-4.979694e+15,8.596004e+15,-5.665159e+17,5.910160e+17,2.535895e+17,1.016657e+16


In [34]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = matrPhi_l[0].dot(matrQ_il[0][l]).dot(la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
print(la.norm(matrPhi_l[25], np.inf))

LinAlgError: singular matrix

In [33]:
matr_a = - copy.deepcopy(matrQ_il[0][0])
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1
matr_a = np.transpose(matr_a)
p0 = la.solve(matr_a, matr_b).reshape(-1)

print(p0)
print(p0.sum())

LinAlgError: singular matrix

In [31]:
vect_p_l = [p0]
for l in range(1, p_num):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(vect_p_l[l])

NameError: name 'p0' is not defined

In [32]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-28.110725,0.000000,20.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
1,20.000000,-28.110725,0.00000,20.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
2,0.000000,0.000000,-20.26325,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
3,0.000000,0.000000,20.00000,-20.26325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
4,5.582250,0.000000,0.00000,0.00000,-15.692975,0.000000,0.000000,0.000000,2.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,440.998200,0.000000,0.00000,0.00000,0.000000,-15.692975,0.000000,0.000000,0.00000,2.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.582250,0.000000,0.00000,0.00000,2.000000,0.000000,-15.692975,0.000000,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,440.998200,0.000000,0.00000,0.00000,0.000000,2.000000,0.000000,-15.692975,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,5.58225,0.00000,0.000000,0.000000,0.000000,0.000000,-443.26145,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,440.99820,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,-443.26145,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
matr_a = np.transpose(matrGamma)
matr_b = np.zeros((matr_a.shape[0], 1))
x = la.lstsq(matr_a, matr_b)[3]
vect_e = np.array([[1.] for i in range(0, x.shape[0])])
res = x.dot(vect_e)[0]
x /= res
print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

x =  [ 0.16664882  0.13778178  0.09753639  0.09537417  0.09494841  0.04665250
  0.04465977  0.04342233  0.04276691  0.03395629  0.03158904  0.03007703
  0.02191321  0.02187875  0.02159697  0.02112990  0.01512927  0.01121088
  0.00796851  0.00373772  0.00334295  0.00324881  0.00142412  0.00060284
  0.00058718  0.00052926  0.00025505  0.00003113]
x1 =  [ 0.16664882  0.13778178  0.09753639  0.09537417]
x2 =  [ 0.09494841  0.04665250  0.04465977  0.04342233  0.04276691  0.03395629
  0.03158904  0.03007703]
x3 =  [ 0.02191321  0.02187875  0.02159697  0.02112990  0.01512927  0.01121088
  0.00796851  0.00373772]


In [ ]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

In [ ]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

In [ ]:
print(la.norm(np.eye(matrQ_il[0][0].shape[0]).dot(matrQ_il[0][1]).dot(la.inv(-matrQ_il[1][1])), np.inf))

In [ ]:
result = np.sum(matrQv_0[0])
i = 0
for matrQk in matrQ_k:
    if i > 0:
        result += np.sum(matrQk[0])
    i += 1
print(result)